# 02-Geocoding

After preparing Trentino school's data, we still need to gather the geographical coordinates of some schools. The reason why aprilascuola and vivoscuola dataset were merged is because schools' coordinates were present in aprilascuola's, which saves us from geocoding the address of over 700 schools.

In [1]:
from shapely.geometry import Point
import geopandas as gpd
import pandas as pd

In [2]:
# Data import
df = pd.read_pickle("../data/trentino/scuole.pkl")
df['CAP'] = df['CAP'].astype('string')

Let's convert the pickle file with schools details to a Geodataframe, considering lon and lat columns as its coordinates:

In [3]:
df = gpd.GeoDataFrame(df,
                      crs='EPSG:4326',
                      geometry=gpd.points_from_xy(df.lon, df.lat))

In [4]:
df.head()

,Id Istituto,Id,Nome,lat,lon,Istituto,Tipo Istituto,Gestione,Indirizzo,Comune,Telefono,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR,CAP,geometry
0,0220629599,0220629699,Settore Industria E Artigianato,46.369265,11.031445,Centro Formazione Professionale Enaip - Cles,Formazione professionale,Paritaria,"Via Mitterer, 10",Cles,0463/421362,0463/421606,cfp.cles@pec.it,cfp.cles@enaip.tn.it,www.enaiptrentino.it,TNFP251STA,38023,POINT (11.03144 46.36927)
1,0221319599,0221319699,Settore Servizi,46.310258,10.745046,Centro Formazione Professionale Enaip - Ossana,Formazione professionale,Paritaria,"Via Di S Antonio, 1",Ossana,0463/751102,0463/751987,cfp.ossana@pec.it,cfp.ossana@enaip.tn.it,www.enaiptrentino.it,TNFP252STA,38026,POINT (10.74505 46.31026)
2,0222049599,0222049699,Settore Industria E Artigianato,46.176075,11.833824,Centro Formazione Professionale Enaip - Primiero,Formazione professionale,Paritaria,"Via Forno - Transacqua, 12",Primiero San Martino Di Castrozza,0439/762057,0439/762833,cfp.primiero@pec.it,cfp.primiero@enaip.tn.it,www.enaiptrentino.it,TNFP253STA,38054,POINT (11.83382 46.17608)
3,0222049599,0222049698,Settore Servizi,46.176075,11.833824,Centro Formazione Professionale Enaip - Primiero,Formazione professionale,Paritaria,"Via Forno - Transacqua, 12",Primiero San Martino Di Castrozza,0439/762057,0439/762833,cfp.primiero@pec.it,cfp.primiero@enaip.tn.it,www.enaiptrentino.it,TNFP253STA,38054,POINT (11.83382 46.17608)
4,0221969599,0221969698,Settore Industria E Artigianato,46.287118,11.511975,Centro Formazione Professionale Enaip - Tesero,Formazione professionale,Paritaria,"Via Caltrezza, 13",Tesero,0462/813133,0462/813145,cfp.tesero@pec.it,cfp.tesero@enaip.tn.it,www.enaiptrentino.it,TNFP254STA,38038,POINT (11.51198 46.28712)


If we consider the rows of the geodataframe were lat or lon are missing, we get the schools with missing coordinates, which are 308.

In [5]:
# Inspecting those schools with no coordinates
geocode_df = df[df['lat'].isna()]

In [6]:
len(geocode_df)

308

In order to geocode the addresses into a shapely Point, we need to compose the complete address of the school. Let's start using nominatim provider on schools' name, address, CAP, municipality and Trento province.  

In [7]:
# Create a complete address with street name and number, cap, municipality and province
tn_addresses = list(geocode_df['Nome']+", "+geocode_df['Indirizzo'].str.title() + ", " +
                    geocode_df['CAP']+", " + geocode_df['Comune'].str.title() + ", Trento")

In [8]:
# Geocoding with OpenStreetMap first, specifying the school to gather the exact position
geo_tn = gpd.tools.geocode(tn_addresses,
                           provider="nominatim",
                           user_agent="schools")

We can merge the dataset with missing values(i.e. `geocode_df`) with the one that detains geocoded coordinates (i.e. `geo_tn`). Whenever the address in `geo_tn` is null, the provider did not find any match with the concatenation of data we gave it. When merging `geocode_df` and `geo_tn`, we add an x-suffix to distinguish the common columns from one another. The merge is not based on some column, but on the row index, which should just concatenate horizontally the two datasets, providing missing geographical information to `geocode_df`. 

We will only consider the rows whose address found a match during the geocoding:

In [9]:
first = geocode_df.merge(geo_tn[~geo_tn['address'].isnull()],
                         left_index=True, right_index=True, suffixes=('x', ''))
first.drop(['geometryx'], axis=1, inplace=True)

In [10]:
first

,Id Istituto,Id,Nome,lat,lon,Istituto,Tipo Istituto,Gestione,Indirizzo,Comune,Telefono,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR,CAP,geometry,address
60,0221612005,0221612113,"Scuola Primaria ""La Vela"" Rovereto",NaN,NaN,Scuola Paritaria La Vela - Fondazione Famiglia...,Scuola Primaria,Paritaria,"Via Saibanti, 6",Rovereto,0464/434047,NaN,segreteria@lavela.famigliamaterna.it,segreteria@lavela.famigliamaterna.it,www.scuolaveronesi.it,TN1E00700T,38068,POINT (11.21984 45.99354),"Scuola dell'infanzia di Vattaro, 7, Via Giardi..."
61,0221612005,0221613050,"Scuola Secondaria Di Primo Grado ""La Vela"" Rov...",NaN,NaN,Scuola Paritaria La Vela - Fondazione Famiglia...,Scuola Secondaria di Primo Grado,Paritaria,"Via Saibanti, 6",Rovereto,0464/435746,NaN,amministrazione@scuolalavela.it,segreteria@lavela.famigliamaterna.it,www.scuolaveronesi.it,TN1M00900P,38068,POINT (11.03047 46.23706),"Scuola dell'Infanzia di Sporminore ""Fratelli R..."
75,0221612907,0221612103,Scuola Primaria Arcivescovile Dame Inglesi Rov...,NaN,NaN,Collegio Arcivescovile Dame Inglesi - Rovereto,Scuola Primaria,Paritaria,"Corso Bettini, 31",Rovereto,0464/406000,0464/406077,collegioarcivescovile@pec.it,lia@arcivescoviletrento.it,www.arcivescoviletrento.it,TN1E006002,38068,POINT (11.06674 45.97856),"scuola dell'infanzia di Cimone Mamma Teresa, 3..."
179,0222059599,0222059699,Settore Servizi,NaN,NaN,Centro Formazione Professionale Upt - Trento,Formazione professionale,Paritaria,"Via Borsieri, 2",Trento,0461/239997,0461/260235,cfp-upt@pec.it,segreteria.sede@cfp-upt.it,www.cfp-upt.it,TNCF003001,38122,POINT (11.15177 46.06472),"Scuola dell'infanzia di Povo, 1, Via Panté, Pa..."


We can replace geometry, lat and long columns values inside the original dataframe with the ones found by geocoding the addresses of schools with missing data. 

In [11]:

df.loc[first.index, 'geometry'] = first['geometry']
df.loc[first.index, 'lat'] = [i.y for i in first['geometry']]
df.loc[first.index, 'lon'] = [i.x for i in first['geometry']]

Still, there are some schools missing. This time, the school's name is not considered in the address, hoping to find a match based on the mere address. The choice of recurring to schools names before was due to a more precise geocoding on OpenStreetMap, which sometimes may not find the right coordinates because schools denominations may differ for some characters between our dataset and data held in OSM. Also, let's try changing provider with Arcgis:

In [12]:
# Geocode with Arcgis the remaining addresses
geocode_df = df[df['lat'].isna()]
# Create a complete address with street name and number, cap, municipality and province
tn_addresses = list(geocode_df['Indirizzo'].str.title() + ", " +
                    geocode_df['CAP']+", " + geocode_df['Comune'].str.title() + ", Trento")

In [13]:
# Geocoding with Arcgis without school names
geo_tn = gpd.tools.geocode(tn_addresses,
                           provider="arcgis")

Still, we consider properly geocoded all those schools with no missing address or no address starting with its CAP (it means it has found the municipality, but not the specific location we're looking for). Notice that CAPs in Trentino all start with 38. 

In [14]:
geo_tn.index = geocode_df.index
second = geocode_df.merge(geo_tn[(~geo_tn['address'].isnull()) &
                                 (~geo_tn['address'].str.startswith("38"))],
                          left_index=True, right_index=True, suffixes=('x', ''))
second.drop(['geometryx'], axis=1, inplace=True)

In [16]:
second

,Id Istituto,Id,Nome,lat,lon,Istituto,Tipo Istituto,Gestione,Indirizzo,Comune,Telefono,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR,CAP,geometry,address
11,0222059596,0222059696,Settore Industria E Artigianato,NaN,NaN,Centro Formazione Professionale Pavoniano Arti...,Formazione professionale,Paritaria,"Piazza Di Fiera, 4",Trento,0461/270244,0461/270241,NaN,scuolagrafica@artigianelli.tn.it,http://www.artigianelli.tn.it/,TNCF004001,38122,POINT (11.12407 46.06439),"Piazza di Fiera 4, 38122, Trento"
13,0221619598,0221619698,Settore Servizi,NaN,NaN,Centro Formazione Professionale Opera Armida B...,Formazione professionale,Paritaria,"Via Setaioli, 5",Rovereto,0464/433771,0464/431711,cfprovereto@pec.operaarmidabarelli.org,cfprovereto@operaarmidabarelli.org,www.operaarmidabarelli.org,TNCF005001,38068,POINT (11.04219 45.88719),"Via Setaioli 5, 38068, Rovereto, Trento"
16,0221539598,0221539698,Settore Servizi,NaN,NaN,Centro Formazione Professionale Upt - Arco,Formazione professionale,Paritaria,"Via Gazzoletti, 10",Arco,0464/556585,0464/556599,cfp-upt@pec.it,segreteria.arco@cfp-upt.it,www.cfpupt.it/sede-di-arco,TNFP261STA,38062,POINT (10.89410 45.91954),"Via Antonio Gazzoletti 10, 38062, Arco, Trento"
74,0221612907,0221613006,Scuola Secondaria Di Primo Grado Arcivescovile...,NaN,NaN,Collegio Arcivescovile Dame Inglesi - Rovereto,Scuola Secondaria di Primo Grado,Paritaria,"Corso Bettini, 31",Rovereto,0464/406000,0464/406077,collegioarcivescovile@pec.it,lia@arcivescoviletrento.it,www.arcivescoviletrento.it,TN1M00200X,38068,POINT (11.04390 45.89239),"Corso Angelo Bettini 31, 38068, Rovereto, Trento"
76,0221612907,0221617115,Liceo Linguistico Arcivescovile Dame Inglesi -...,NaN,NaN,Collegio Arcivescovile Dame Inglesi - Rovereto,Scuola Secondaria di Secondo Grado,Paritaria,"Corso Bettini, 71",Rovereto,0464/406000,0464/406077,collegioarcivescovile@pec.it,lia@arcivescoviletrento.it,www.arcivescoviletrento.it,TNPL005006,38068,POINT (11.04333 45.89687),"Corso Angelo Bettini 71, 38068, Rovereto, Trento"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,NaN,NaN,Scuola Dell'Infanzia Di Trento Piedicastello,NaN,NaN,Circolo Di Coordinamento N. 5,Scuola dell'Infanzia,Statale,"Via Dos Trento, 86",Trento,0461/983539,0461/497257,scuolainfanzia.piedicastello@scuole.provincia....,circolo.coordinamento05@provincia.tn.it,NaN,NaN,38122,POINT (11.11417 46.07174),"Via Dos Trento 86, 38121, Trento"
720,NaN,NaN,"Scuola Dell'Infanzia Di Tierno Mori ""Il Giras...",NaN,NaN,Circolo Di Coordinamento N. 9,Scuola dell'Infanzia,Statale,"Via della Cooperazione, 20",Mori,0464/917200,0464/493117,scuolainfanzia.tierno@scuole.provincia.tn.it,circolo.coordinamento09@provincia.tn.it,NaN,NaN,38065,POINT (10.97992 45.84566),"Via della Cooperazione 20, 38065, Mori, Trento"
721,NaN,NaN,"Scuola Dell'Infanzia Di Pellizzano ""Antonio Bo...",NaN,NaN,Circolo Di Coordinamento N. 11,Scuola dell'Infanzia,Statale,"Via Dei Baschenis, 3",Pellizzano,335 1817040,0461/491940,scuolainfanzia.pellizzano@scuole.provincia.tn.it,circolo.coordinamento11@provincia.tn.it,NaN,NaN,38020,POINT (10.75920 46.30963),"Via dei Baschenis 3, 38020, Pellizzano, Trento"
722,NaN,NaN,Scuola Dell'Infanzia Di Tesero,NaN,NaN,Scuola Dell'Infanzia Di Tesero,Scuola dell'Infanzia,Paritaria,"Via Fia, 20",Tesero,0462-813045,NaN,tesero.presidente@fpsm.tn.it,tesero.segretario@fpsm.tn.it,NaN,NaN,38038,POINT (11.51497 46.28956),"Via Fia 20, 38038, Tesero, Trento"


We can replace geometry, lat and long columns values inside the original dataframe with the ones found by geocoding the addresses of schools with missing data. 

In [17]:
df.loc[second.index, 'geometry'] = second['geometry']
df.loc[second.index, 'lat'] = [i.y for i in second['geometry']]
df.loc[second.index, 'lon'] = [i.x for i in second['geometry']]

In [18]:
geocode_df = df[df['lat'].isna()]

In [19]:
len(geocode_df)

24

Still, 24 schools' coords are missing. Let's retry without the CAP column and by using nominatim provider:

In [20]:
tn_addresses = list(geocode_df['Indirizzo'].str.title() + ", " +
                    geocode_df['Comune'].str.title() + ", Trento")

In [21]:
# Geocoding with OpenStreetMap without school names
geo_tn = gpd.tools.geocode(tn_addresses,
                           provider="nominatim",
                           user_agent="schools")

As before, let's remove the non-matching addresses and merge the missing schools' data with the matching coordinates found:

In [22]:
geo_tn.index = geocode_df.index
third = geocode_df.merge(geo_tn[~geo_tn['address'].isnull()],
                          left_index=True, right_index=True, suffixes=('x', ''))
third.drop(['geometryx'], axis=1, inplace=True)

In [24]:
df.loc[third.index, 'geometry'] = third['geometry']
df.loc[third.index, 'lat'] = [i.y for i in third['geometry']]
df.loc[third.index, 'lon'] = [i.x for i in third['geometry']]

Despite the three geocoding with different combinations of fields to create the address and different providers, some schools seem they don't want to be found 😅. Some of these schools were searched and inserted inside OpenStreetMap, but still the provider has some difficulties in finding these schools. 

In [25]:
# Search for None objects remaining
geocode_df = df[df['lat'].isna()]

In [26]:
geocode_df

,Id Istituto,Id,Nome,lat,lon,Istituto,Tipo Istituto,Gestione,Indirizzo,Comune,Telefono,Fax,Email istituto,Email segreteria,Sito web,Codice MIUR,CAP,geometry
525,NaN,NaN,"Scuola Dell'Infanzia Di Pera Di Fassa ""Don Edy...",NaN,NaN,Coordinamento Pedagogico Scuole Provinciali De...,Scuola dell'Infanzia,Statale,Piaza don Giuseppe Antonio Vian - Pozza di Fas...,San Giovanni Di Fassa - Sen Jan,0462/764828,0462/760001,scuolainfanzia.pera@scuole.provincia.tn.it,coordinamento.scuoleladine@provincia.tn.it,NaN,NaN,38036,POINT EMPTY
545,NaN,NaN,Scuola Dell'Infanzia Di Cimego,NaN,NaN,Circolo Di Coordinamento N. 8,Scuola dell'Infanzia,Statale,"Via don Bertolasi - Cimego, 64",Borgo Chiese,0465/621567,0461/493389,scuolainfanzia.cimego@scuole.provincia.tn.it,circolo.coordinamento08@provincia.tn.it,NaN,NaN,38083,POINT EMPTY
571,NaN,NaN,Scuola Dell'Infanzia Di Molina Di Ledro,NaN,NaN,Scuola Dell'Infanzia Di Molina Di Ledro,Scuola dell'Infanzia,Paritaria,"Via Sartori - Molina di Ledro, 25",Ledro,0464-508477,NaN,molina.ledro.presidente@fpsm.tn.it,molina.ledro.segretario@fpsm.tn.it,NaN,NaN,38067,POINT EMPTY
605,NaN,NaN,Scuola Dell'Infanzia Di Ponte Arche,NaN,NaN,Scuola Dell'Infanzia Di Ponte Arche,Scuola dell'Infanzia,Paritaria,Via S Giovanni Bosco - Ponte Arche - Bleggio I...,Comano Terme,0465-702422,NaN,ponte.arche.presidente@fpsm.tn.it,ponte.arche.segretario@fpsm.tn.it,NaN,NaN,38070,POINT EMPTY
626,NaN,NaN,Scuola Dell'Infanzia Di Madonna Di Campiglio,NaN,NaN,Scuola Dell'Infanzia Di Madonna Di Campiglio,Scuola dell'Infanzia,Paritaria,"Via Serafini - Madonna di Campiglio, 13",Tre Ville,0465-442779,0465 442779,madonna.campiglio.segretario@fpsm.tn.it,madonna.campiglio.segretario@fpsm.tn.it,NaN,NaN,38070,POINT EMPTY
659,NaN,NaN,Scuola Dell'Infanzia Di Serravalle,NaN,NaN,Scuola Dell'Infanzia Di Serravalle,Scuola dell'Infanzia,Paritaria,"Via Ingenger Tomasi - Serravalle, 5",Ala,0464-696304,NaN,serravalle.presidente@fpsm.tn.it,serravalle.segretario@fpsm.tn.it,NaN,NaN,38061,POINT EMPTY
666,NaN,NaN,Scuola Dell'Infanzia Di S. Croce Bleggio,NaN,NaN,Scuola Dell'Infanzia Di S. Croce Bleggio,Scuola dell'Infanzia,Paritaria,"Frazione Santa Croce di Bleggio Inferiore, 7",Comano Terme,0465-779900,NaN,s.croce.presidente@fpsm.tn.it,s.croce.segretario@fpsm.tn.it,NaN,NaN,38070,POINT EMPTY
668,NaN,NaN,Scuola Dell'Infanzia Di Vigo Di Fassa,NaN,NaN,Scuola Dell'Infanzia Di Vigo Di Fassa,Scuola dell'Infanzia,Paritaria,"Strada Giuseppe Soraperra - Pozza di Fassa, 4",San Giovanni Di Fassa - Sen Jan,0462-763194,NaN,vigo.fassa.presidente@fpsm.tn.it,vigo.fassa.segretario@fpsm.tn.it,NaN,NaN,38036,POINT EMPTY


Considering that the remaining schools are 8, their coordinates are inserted manually. 

In [27]:
# Manually inserting coordinates
geocode_df[['lat','lon']] = [[46.43982, 11.69179],
                             [45.91263, 10.61412],
                             [45.87063, 10.77462],
                             [46.0354, 10.87076],
                             [46.22257, 10.82716],
                             [45.81072, 11.01411],
                             [46.02616, 10.83888],
                             [46.42304, 11.68182]]
geocode_df['geometry'] = geometry = [Point(xy) for xy in zip(geocode_df['lon'], geocode_df['lat'])]

C:\Users\auror\anaconda3\envs\geospatial\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
C:\Users\auror\anaconda3\envs\geospatial\lib\site-packages\geopandas\geodataframe.py:1351: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


For the last time, let's replace missing values with the ones manually inserted:

In [28]:
df.loc[geocode_df.index, 'geometry'] = geocode_df['geometry']
df.loc[geocode_df.index, 'lat'] = [i.y for i in geocode_df['geometry']]
df.loc[geocode_df.index, 'lon'] = [i.x for i in geocode_df['geometry']]

We can check if there are missing shapely Points by calling `df.explore()`, since it will not execute if some None points are found. Also, we can inspect schools on the map and see if some points are outside the perimetered territory of Trentino.

In [29]:
# Check if there are None points
df.explore()

In the end, we can save these data, both as geojson and shapefile (which will be necessary for the R analysis).

In [30]:
# Saving coordinates in GeoJSON
df.to_crs(4326).to_file("../data/trentino/schools/schools.geojson", index=False)

# Saving schools as shapefile
df = gpd.read_file("../data/trentino/schools/schools.geojson")
df.to_file("../data/Trentino/schools", driver='ESRI Shapefile')

C:\Users\auror\AppData\Local\Temp/ipykernel_35376/3060867726.py:6: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  df.to_file("../data/Trentino/schools", driver='ESRI Shapefile')
